In [0]:
%sql
CREATE DATABASE IF NOT EXISTS tfm_config
COMMENT 'Base de configuración del Data Lakehouse. Contiene reglas y catálogos auxiliares para la normalización, mapeo y tipificación de columnas en las capas Bronze y Silver.';

In [0]:
%sql
-- ==========================================================================
-- TABLA: tfm_config.header_mappings
-- Descripción general:
-- Esta tabla mantiene el catálogo de mapeos entre los nombres de columnas
-- originales (header_source) y los nombres canónicos estandarizados
-- (header_canonical). 
--
-- Su propósito es garantizar que los encabezados de los archivos fuente
-- provenientes de distintos años o versiones se transformen correctamente
-- a una estructura unificada en la capa Silver.
--
-- Permite además manejar la vigencia temporal de cada mapeo y resolver
-- conflictos por prioridad o año específico.
-- ==========================================================================
--DROP TABLE IF EXISTS tfm_config.header_mappings;
CREATE TABLE IF NOT EXISTS tfm_config.header_mappings (
  dataset_id        STRING COMMENT 'Identificador lógico del conjunto de datos al que aplica el mapeo (por ejemplo: "vehiculos"). Permite reutilizar esta tabla para múltiples datasets.',
  header_source     STRING COMMENT 'Nombre original de la columna en el archivo fuente, ya sanitizado (sin acentos, en mayúsculas)',
  header_canonical  STRING COMMENT 'Nombre canónico o estandarizado al que debe transformarse el encabezado fuente. Ejemplo: "CODIGO_VEHICULO", "FECHA_PROCESO".',
  effective_start   TIMESTAMP COMMENT 'Fecha y hora a partir de la cual el mapeo es vigente (inclusive). Facilita control de versiones y evolución del esquema.',
  effective_end     TIMESTAMP COMMENT 'Fecha y hora hasta la cual el mapeo es válido (exclusiva). NULL indica que el mapeo sigue vigente.',
  source_year       INT COMMENT 'Año específico del dataset al que aplica la regla (por ejemplo, 2017). Si es NULL, la regla aplica a todos los años.',
  priority          INT COMMENT 'Nivel de prioridad en caso de duplicidad de mapeos. Un número menor indica mayor prioridad (más relevante).',
  note              STRING COMMENT 'Campo descriptivo para incluir observaciones, referencias o justificación del mapeo aplicado.'
)
USING DELTA
COMMENT 'Catálogo de correspondencia entre encabezados originales de los archivos y nombres canónicos estandarizados. Permite control temporal y por año de las reglas de mapeo aplicadas en el pipeline de ingesta y normalización.';